In [1]:
# Copyright 2020 Bimghi Choi. All Rights Reserved.
# 예측 + 투자전략 시스템

# _*_ coding: utf-8 _*_

from tensorflow import keras
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]='0, 1'

In [2]:
import util
import preprocess as prepro
import models
import learn
from learn import GenerateResult
import math
from datetime import datetime

In [3]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [4]:
#tf.keras.backend.set_floatx('float64')

In [5]:
import gc
gc.collect()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')    
    #tf.config.experimental.set_visible_devices(gpus[1], 'GPU')    
#    tf.config.experimental.set_virtual_device_configuration(
#        gpus[0],
#        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    #tf.config.experimental.set_virtual_device_configuration(
    #    gpus[1],
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])    
  except RuntimeError as e:
    # 프로그램 시작시에 가장 장치가 설정되어야만 합니다
    print(e)

In [6]:
file_name = '~/Data/kospi200f_809_0515.csv'
item_name = 'kospi200f_loss_cut'
train_start = '2000-01-31'
train_end = '2017-01-02'
test_start = '2017-01-02'
test_end = '2020-05-15'

remove_columns = ['date', '종가']
target_column = '종가'
input_columns = []
target_type = 'rate'

model_name = 'bayesian-opt'
channel = False

trans_day = 1

target_alpha = 100
future_day = 2
train_end_back = -1
n_timestep = 30
time_interval = 1
input_size = 809
n_unit = 1024
batch_size = 10
learning_rate = 0.0005
epochs = 3

alpha = 0.05
beta = 0.01

comment = "2일 예측을 이용 1일 예측, bayesian optimization (timestep, interval, units, batch_size,  epochs) loss - model1"

checkpoint_path = model_name + "/pred"+str(future_day)+"-"+datetime.now().strftime("%Y-%m-%d:%H:%M:%S")+".ckpt"

In [7]:
dataframe = util.read_datafile(file_name)
df = dataframe.copy()
#df = prepro.target_conversion(df, target_column, future_day, type=target_type)
#a = []
#b = []
for i in range(len(df[target_column]) -future_day):
    df.loc[i, target_column] = ((df.loc[i + future_day, target_column] - df.loc[i, target_column]) 
                                / df.loc[i, target_column]) * target_alpha 
    df.loc[i, '시가'] = df.loc[i + future_day + train_end_back + 1, '시가']
#    a.append(max(df.loc[i + future_day + train_end_back + 1:i + future_day + 1, '고가']))
#    b.append(min(df.loc[i + future_day + train_end_back + 1:i + future_day + 1, '저가']))
#for i in range(len(df[target_column]) -future_day):
#    df.loc[i, '고가'] = a[i]
#    df.loc[i, '저가'] = b[i]
    

In [8]:
df.head(20)

,date,시가지수(포인트),고가지수(포인트),저가지수(포인트),종가지수(포인트),수익률(%),수익률 (1주)(%),수익률 (1개월)(%),수익률 (3개월)(%),수익률 (6개월)(%),...,"주요상품선물_천연가스(선물, NYMEX)($/mmBtu)",주요상품선물_금(선물)($/ounce),주요상품선물_은(선물)($/ounce),주요상품선물_알루미늄(선물)($/ton),주요상품선물_옥수수(최근월물)(￠/bu),대두박(￠/bu),종가,시가,고가,저가
0,2000-01-31,-0.218225,-0.493106,-0.400439,-0.223565,0.212872,1.352924,-0.851790,-0.382658,-0.162615,...,1.653438,-0.853010,0.974513,0.827681,0.373581,0.614542,-0.961538,116.55,120.10,116.65
1,2000-02-01,-0.213200,-0.230417,-0.373954,-0.595373,-0.407635,1.808646,-1.074203,-1.707405,-0.490731,...,1.687061,-0.039689,0.193571,1.002058,0.594935,0.706918,3.381014,119.35,121.70,115.35
2,2000-02-02,-0.208520,-0.448886,-0.367787,-0.200398,0.696677,2.107257,-0.769928,-1.558400,0.074389,...,1.809524,0.800355,0.031266,0.069626,0.430159,0.725768,3.292528,120.40,119.70,115.95
3,2000-02-03,-0.204125,-0.216603,-0.062403,-0.021445,0.366314,1.488978,-0.625155,-1.268930,-0.079967,...,1.213642,1.515992,0.623370,0.378121,0.084769,0.352875,1.677149,122.90,120.25,116.80
4,2000-02-07,-0.200000,0.378008,0.426748,0.592789,0.949260,1.101574,1.281908,-1.288431,1.273730,...,0.689954,3.856851,0.568635,-0.322023,0.518339,0.702038,0.653862,122.40,123.45,119.75
5,2000-02-08,-0.196110,0.353748,0.677387,0.254527,-0.366454,1.138729,0.998930,-1.329397,0.942469,...,0.332052,2.657684,1.222974,-0.720402,0.474125,0.440887,1.319588,120.85,123.60,120.95
6,2000-02-09,-0.192447,0.306497,0.819824,0.658684,0.641075,1.076862,1.233568,-1.637689,0.994675,...,0.557674,3.026664,2.114708,-0.407511,0.622228,0.721716,-1.867641,123.60,124.10,121.85
7,2000-02-10,0.330923,0.617187,0.552762,0.366383,-0.311193,0.651653,0.275024,-1.607973,0.269679,...,0.812787,3.316674,1.871659,-0.157431,0.985272,1.379493,-5.575906,118.85,125.05,120.40
8,2000-02-11,0.977388,0.576306,0.048738,-0.013448,-0.428351,0.326747,0.131530,-1.828185,0.546650,...,0.675096,2.384344,0.988933,-0.154629,0.473525,0.907189,-8.274721,116.45,124.75,119.85
9,2000-02-14,-0.649462,-0.806483,-0.954529,-1.222378,-1.533955,-1.217328,-0.075927,-2.486079,0.140712,...,0.504701,1.935983,0.085505,-0.246361,0.529501,0.923675,-1.422414,112.05,119.00,115.25


In [9]:
train_start_index = max(df.loc[df['date']<=train_start].index) + time_interval*(n_timestep-1) - 1
train_end_index = max(df.loc[df['date']<=train_end].index)
base_prices = tf.reduce_mean(df.loc[train_start_index:train_end_index+1, '시가'])  

@tf.function
def loss_fn_model1_1(targets, preds):
 
    loss0 = tf.keras.losses.MSE(targets, preds)
    
    preds = tf.reshape(preds[:, -1, :], [-1])
    targets = tf.reshape(targets[:, -1, :], [-1])
    
   
    if alpha != 0:
        # add RRL cost - maximize downside sharp ratio

        # 1 if (pred - base) * (target - base) > 0, -1 otherwise
        F = tf.math.sign(targets*preds)
        F = tf.reshape(F, [-1])

        # calc returns from each step in batches
        R = tf.math.divide(tf.math.multiply(tf.math.abs(targets), (F - 0.00003)), base_prices)
        R = tf.reshape(R, [-1])

        # calc downside sharp ratio

        # downside returns
        DR = tf.minimum(0.0, R)
        DR = tf.reshape(DR, [-1])

        # calc. downside sharp ratio
        #s = []
        #for i in range(batch_size):
        #   std =  tf.keras.backend.std(DR[i, :, 0])
        #   s.append(tf.reduce_mean(R[i, :, 0])/tf.maximum(0.01, std))

        # calc. downside sharp ratio
        loss1 = tf.reduce_mean(R) / (tf.keras.backend.std(DR) + 0.001)
    else:
        loss1 = 0

    """
    # average profits, loss
    avg_plusR = [0.0]
    avg_minusR = [0.0]

    global num_of_profits
    global num_of_losses

    num_of_profits = 0
    num_of_losses = 0

    for i in range(batch_size):
        res = tf.cond(R[i, num_steps - 1, 0] > 0, lambda: return_one(), lambda: return_zero())
        if res == 1:
            avg_plusR.append(R[i, num_steps - 1, 0])
        else:
            avg_minusR.append(R[i, num_steps - 1, 0])
    avg_profit = tf.reduce_mean(avg_plusR) 
    avg_loss = tf.reduce_mean(avg_minusR) 
    """

    if beta != 0:
        #compute maximum drawdown

        #accm_profit = [0.0]
        #for i in range(batch_size):
        #    for j in range(num_steps):
        #        r = tf.cond((predict_prices[i, num_steps-1, 0] - base_prices[i, num_steps-1, 0]) *
        #                   (target_prices[i, num_steps-1, 0] - base_prices[i, num_steps-1, 0]) > 0,
        #                   lambda: return_one(),
        #                   lambda: return_zero())
        #        if r == 1: accm_profit.append(accm_profit[i*num_steps + j] + tf.abs(target_prices[i, j, 0] - base_prices[i, j, 0]))
        #        else:      accm_profit.append(accm_profit[i*num_steps + j] - tf.abs(target_prices[i, j, 0] - base_prices[i, j, 0]))

        accm_profit = [0.0 for i in range(batch_size)]
        for i in range(batch_size):
            if i == 0:
                accm_profit[0] = tf.sign(preds * targets) * tf.math.abs(targets)
            else:
                accm_profit[i] = accm_profit[i-1] + tf.sign(preds * targets) * tf.math.abs(targets)
        loss2 = (tf.reduce_max(accm_profit) - tf.reduce_min(accm_profit))/batch_size
    else:
        loss2 = 0

    return loss0 + beta*loss2 - alpha*loss1

In [10]:
@tf.function
def loss_fn_model1(train_y, preds):
    
    #preds = tf.cast(preds, dtype=tf.float64)
    #train_y = tf.cast(train_y, dtype=tf.float64)
    rates = train_y / 100
    updown = tf.math.sign(train_y)
    profits = 1 + rates*tf.math.sign(preds)
    
    loss1 = keras.losses.MSE(train_y, preds)
    
    batches = tf.shape(train_y)[0]

    return_plus = 0.000000000
    return_minus = 0.000000000
    for i in range(batches):
        if profits[i, -1, 0] - 1 > 0.0: return_plus += profits[i, -1, 0]-1      
        else: return_minus += 1-profits[i, -1, 0]    

    loss2 = (return_plus - return_minus)/tf.cast(batches, dtype=tf.float32)
    
    return loss1 - loss2

In [11]:
accus = []
predictions = []

def objective_function(args):
    
    print("실험 시작\nargs: {}".format(args))
    print(datetime.now().strftime("%Y-%m-%d:%H:%M:%S"))
    n_timestep = int(args[0, 0])
    time_interval = int(args[0, 1])
    n_units = int(args[0, 2])
    batch_size = int(args[0, 3])
    epochs = int(args[0, 4])
    """
    model1 = tf.keras.Sequential([
        tf.keras.Input(shape=(n_timestep, input_size)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_units, return_sequences=True, kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.01))),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(n_units/2), return_sequences=True, kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.01))),
        tf.keras.layers.Dense(1)
    ])
    """

    accu = np.zeros(2)
    
    for k in range(2):
        
        # 모델 생성
        model = models.LSTM(n_timestep,input_size,n_unit,regularizers_alpha=0.01,drop_rate=0.5)

        model.compile(optimizer='adam',
                      loss=loss_fn_model1)
                          #callbacks=[cp-callback]
                      #metrics=['accuracy'])
        model.save_weights(checkpoint_path)            

        # 초기 train, test start-end date 설정
        current_train_start = df.loc[prepro.date_to_index(df, train_start), 'date']
        current_train_end = df.loc[prepro.date_to_index(df, train_end), 'date']
        current_test_start = df.loc[prepro.date_to_index(df, test_start), 'date']
        current_test_end = df.loc[prepro.date_to_index(df, test_start) + trans_day - 1, 'date']


        #  각 transfer 구간의 예측값들을 합치기 위하여
        test_prediction1 = []

        while True:

            gc.collect()

            # train, test data 분리 생성 - 불칠요한 column 제거, 재배치
            train_data, test_data = prepro.get_train_test_data(df, target_column, remove_columns, 
                                                                   current_train_start, current_train_end,
                                                                   current_test_start, current_test_end,
                                                                   future_day, n_timestep, time_interval)

            # 전체 train, test dataset 생성
            train_x, train_y = prepro.get_LSTM_dataset(train_data, n_timestep, time_interval, input_size, future_day)
            
            # the model1 training dataset
            train_x = train_x[:train_end_back]
            train_y = train_y[:train_end_back]

            # random split
            train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.5)
            
            # test set 생성
            test_x, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)
            test_y_target = np.expand_dims(test_y[:, :, 3], axis=2)            

            # 전체 dataset training
            train_y_target = np.expand_dims(train_y[:, :, 3], axis=2)     
            early_stopping1 = tf.keras.callbacks.EarlyStopping(patience=2, verbose=0)        
            model.load_weights(checkpoint_path)
            model.fit(train_x, train_y_target, verbose=0, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping1], validation_data=(test_x, test_y_target))
            model.save_weights(checkpoint_path)


            #print('test dates ' + current_test_start + "~" + current_test_end)

            # prediction1 accuracy
            prediction1 = model.predict(test_x)[:, -1, 0].reshape(-1)
            test_prediction1.append(prediction1)


            if current_test_end == test_end:
             break

            #train, start dates shift
            current_train_end = df.loc[prepro.date_to_index(df, current_train_end) + trans_day, 'date']
            current_train_start = df.loc[prepro.date_to_index(df, current_train_end) - 2000, 'date']
            current_test_start = df.loc[prepro.date_to_index(df, current_test_start) + trans_day, 'date']
            if prepro.date_to_index(df, test_end) - prepro.date_to_index(df, current_test_start) < trans_day:
                current_test_end = test_end
            else:
                current_test_end = df.loc[prepro.date_to_index(df, current_test_end) + trans_day, 'date']
        #end while
        
        t = np.concatenate(test_prediction1) 
        if k == 0: t0 = t.copy()
            
        # 전체 test_oouput 생성
        _, test_data = prepro.get_train_test_data(df, target_column, remove_columns,
                                                           train_start, train_end,
                                                           test_start, test_end,
                                                           future_day, n_timestep, time_interval)
        _, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

        # test accuracy 계산
        test_start_index = df.loc[df['date'] <= test_start].index.max()
        test_end_index = df.loc[df['date'] <= test_end].index.max()
        test_open_prices = np.array(list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '시가'])))
        test_base_prices = np.array(list(map(float, dataframe.loc[test_start_index - future_day: test_end_index - future_day, '종가'])))
        test_output_prices = np.array(list(map(float, dataframe.loc[test_start_index: test_end_index, '종가'])))   
        test_predict_prices = test_base_prices * (np.array(t)/100 + 1)


        # 예측확률 계산
        cnt = 0
        for i in range(len(test_predict_prices)):
            if (test_predict_prices[i]-test_open_prices[i])*(test_output_prices[i]-test_open_prices[i]) > 0:
                cnt += 1
        accu[k] = cnt/len(test_predict_prices)
        
        print("실험 하나 종료! 결과\nargs: {}, acc: {}, {}".format(args, accu[k], k))
        print(datetime.now().strftime("%Y-%m-%d:%H:%M:%S"))
    #end for
    
    accus.append(max(accu[0], accu[1]))
    if accu[0] > accu[1]: predictions.append(t0)
    else: predictions.append(t)
    
    return 1 - max(accu[0], accu[1])

In [12]:
#!pip install GPyOpt
from GPyOpt.methods import BayesianOptimization
domain = [{'name': 'n_timestep', 'type': 'discrete', 'domain': (10, 15, 20, 25, 30, 35, 40, 45), 'dimensionality': 1},
          {'name': 'time_interval', 'type': 'discrete', 'domain': (1, 2, 3), 'dimensionality': 1},
          {'name': 'n_unit', 'type': 'discrete', 'domain': (128, 256, 500, 1024), 'dimensionality': 1},
          {'name': 'batch_size', 'type': 'discrete', 'domain': (10, 15, 20, 25, 30, 35, 40, 45, 50), 'dimensionality': 1},
          {'name': 'epochs', 'type': 'discrete', 'domain': (3, 5, 7, 10), 'dimensionality': 1},]
accus = []
predictions = []
myBopt = BayesianOptimization(f=objective_function, domain=domain, initial_design_numdata=5)
myBopt.run_optimization(max_iter=15)
myBopt.plot_acquisition

실험 시작
args: [[ 40.   2. 500.  10.   3.]]
2020-06-27:01:28:41


KeyboardInterrupt: 

In [ ]:
print(myBopt.x_opt)  # lr_rate, #layer, hidden_size, batch_size

n_timestep = myBopt.x_opt[0]
time_interval = myBopt.x_opt[1]
n_unit = myBopt.x_opt[2]
batch_sixe = myBopt.x_opt[3]

N, _ = myBopt.X.shape
for i in range(N):
    if np.array_equal(myBopt.x_opt, myBopt.X[i, :]):
        print(1 - myBopt.Y[i,:])  # accuracy 변환

In [ ]:
best_accu_idx = np.argmax(accus)
t = predictions[best_accu_idx]

In [ ]:
future_day = int(future_day)
n_timestep = int(n_timestep)
time_interval = int(time_interval)

In [ ]:
#test_prediction = np.concatenate(test_prediction, axis=0).reshape((-1, n_timestep, 1))
#train_prediction = learn.predict_batch_test(model1, train_x[:batch_size], len(train_x[:batch_size]))

# 원 지수, 날짜는 target conversion이 되기 전 dataframe에서 가져옴
test_dates, test_base_prices, train_dates, train_base_prices = prepro.get_test_dates_prices(dataframe, test_start, test_end,
                                                      train_start, train_end, n_timestep, time_interval, future_day, target_column)

# 전체 test_oouput 생성
_, test_data = prepro.get_train_test_data(df, target_column, remove_columns,
                                                   train_start, train_end,
                                                   test_start, test_end,
                                                   future_day, n_timestep, time_interval)
_, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

In [ ]:
updown = np.sign(test_y[:, -1, 3]).reshape((-1))
#calculate accuracy
temp = tf.math.multiply(updown, t.reshape((-1)))
accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
print('accuracy = ', accu)

In [ ]:
updown.shape

In [ ]:
#train_base_prices = train_base_prices[:batch_size]
#train_prediction = train_prediction[:batch_size]
#train_y = train_y[:batch_size]

In [ ]:
result = GenerateResult(t, t, test_y[:, -1, 3].reshape(-1), test_dates, n_timestep, future_day, train_end_back, trans_day)

In [ ]:
test_start_index = prepro.date_to_index(df, test_start)
test_end_index = prepro.date_to_index(df, test_end)
test_open_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '시가']))
test_high_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '고가']))
test_low_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '저가']))

In [ ]:
#result.extract_last_output()
result.convert_price(test_base_prices,conversion_type=target_type)
# 손익 계산
profits = np.zeros(len(test_dates))
for i in range(len(test_dates)):
    if result.test_predict_price[i]-test_open_prices[i] > 0:
        profits[i] = result.test_output_price[i] - test_open_prices[i]
    else:
        profits[i] = test_open_prices[i] - result.test_output_price[i]

In [ ]:
#result.extract_last_output()
result.convert_price(test_base_prices,conversion_type=target_type)

In [ ]:
import operator
preds = np.array(result.test_predict_price) - np.array(test_open_prices)
trues = np.array(result.test_output_price) - np.array(test_open_prices)
result.evaluation(preds, trues)
result.table(test_open_prices, test_high_prices, test_low_prices, profits, profits)
result.save_result(model_name,item_name,n_unit,target_type,batch_size,n_timestep,time_interval,epochs,str(alpha),comment)
result.save_visualization()
#result.save_model(model)